# 1. Importing Libraries

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Reading the Data

In [3]:
project_dir=r"C:\Users\Dell\Desktop\Filghts prediction Project-sagemaker"
data_dir='data'

In [4]:
os.path.join(project_dir,data_dir,'flight_price.csv')

'C:\\Users\\Dell\\Desktop\\Filghts prediction Project-sagemaker\\data\\flight_price.csv'

In [5]:
def get_data(name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_dir,data_dir,file_name)
    return pd.read_csv(file_path)

In [6]:
flights=get_data('flight_price')
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24-03-2019,Banglore,New Delhi,BLR → DEL,22:20,22-03-2024 01:10,2h 50m,non-stop,No info,3897
1,Air India,01-05-2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,09-06-2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,10-06-2024 04:25,19h,2 stops,No info,13882
3,IndiGo,12-05-2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01-03-2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,09-04-2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27-04-2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27-04-2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01-03-2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contain 10,683 rows and 11 features
- Columns 'Route' and 'Total_Stops' have missing value each
- The data types of some features isn't appripriate

# 3. Preliminary Analysis

### 3.1 Check Data Types

In [8]:
flights.head(2)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24-03-2019,Banglore,New Delhi,BLR → DEL,22:20,22-03-2024 01:10,2h 50m,non-stop,No info,3897
1,Air India,01-05-2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662


In [9]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [10]:
value=flights.Date_of_Journey.iloc[3]
value

'12-05-2019'

### 3.2 Check for Duplicate

In [11]:
flights.duplicated().sum()

220

In [12]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(['Airline','Date_of_Journey','Source','Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01-03-2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,02-03-2024 23:55,39h 5m,2 stops,No info,17135
9848,Air India,01-03-2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,02-03-2024 23:55,39h 5m,2 stops,No info,17135
1495,Air India,01-04-2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,02-04-2024 01:20,15h 20m,2 stops,No info,10408
9913,Air India,01-04-2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,02-04-2024 01:20,15h 20m,2 stops,No info,10408
3598,Air India,01-05-2019,Kolkata,Banglore,CCU → GAU → DEL → BLR,09:50,02-05-2024 08:55,23h 5m,2 stops,No info,13227
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24-03-2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24-03-2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24-03-2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24-03-2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of ' Date_of_Journey ',' Dep_Time ',' Arrival_Time ' should datatime
- The type of ' Duration ' and ' Total_Stops ' is mixed.it should be numeric type
- There are 220 duplicates. These should be removed

# 4. Details Analysis

# Airline

In [13]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of the entries are inconsistent values/inaccurate values

In [14]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()  # cacpitalize first letter
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

In [15]:
flights.Date_of_Journey.head(2)

0    24-03-2019
1    01-05-2019
Name: Date_of_Journey, dtype: object

In [16]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [17]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [18]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [43]:
###  Dept Time
flights.Dep_Time.head(3)

0    22:20
1    05:50
2    09:25
Name: Dep_Time, dtype: object

In [44]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [45]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\Dell\AppData\Local\Temp\ipykernel_13148\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [82]:
(
    flights
    .Arrival_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(1)
)

0        01:10
2        04:25
6        10:25
7        05:05
8        10:25
         ...  
10666    19:00
10667    20:20
10672    19:00
10673    04:25
10674    21:20
Name: Arrival_Time, Length: 4335, dtype: object

In [116]:
# duration
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [117]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('m')]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [119]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('h')]
    #.unique()
)

6474    5m
Name: Duration, dtype: object

In [120]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,06-03-2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The Obervation indexed 6474 has duration of 5 minutes. This is clearly wrong .will delete this obervation 

In [137]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour=lambda df_: (
        df_
        .hour
        .str.replace("h","")
        .astype(int)
        .mul(60)
        ),
        minute=lambda df_:(
        df_
        .minute
        .str.replace("m",""))
        .fillna("0")
        .astype(int)
    )
    #.dtypes
    #isna().sum()
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [139]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour=lambda df_: (
        df_
        .hour
        .str.replace("h","")
        .astype(int)
        .mul(60)
        ),
        minute=lambda df_:(
        df_
        .minute
        .str.replace("m",""))
        .fillna("0")
        .astype(int)
    )
    #.dtypes
    #isna().sum()
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [147]:
# Total_stopms

flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [151]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)  # serarch for " stop" or " stops"
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [157]:
# Additional info

flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [19]:
flights.Airline.value_counts()

Airline
Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64

# 5. Cleaning Operations

In [21]:
# df['colum_name']=df['column_name']+value

In [22]:
# To lower case the column name 
# columns=[col.lower() for col in flights.columns ]
# flights.columns= columns

In [23]:
# to Remove white space from strings
# flights['colum_name']=df['column_name'].str.strip()

In [24]:
# extracting object type columns names
# flights.select_dtypes(include="O").columns

In [142]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ",expand=True)
        .set_axis(["hour","minute"],axis=1)
        .assign(
            hour=lambda df_: (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
            minute=lambda df_:(
            df_
            .minute
            .str.replace("m",""))
            .fillna("0")
            .astype(int)
    )
    #.dtypes
    #isna().sum()
    .sum(axis=1)
)


In [162]:
def clean_data(df):
    return(
        df
        .drop(index=[6474])
        .assign(**{col: df[col].str.strip()
                  for col in df.select_dtypes(include="O").columns
                  })
        .rename(columns=str.lower)
        .assign(airline=lambda df_:(
            flights
            .Airline
            .str.replace(" Premium economy","")
            .str.replace(" Business","")
            .str.title()  # cacpitalize first letter
        ),
                date_of_journey=lambda df_: pd.to_datetime(df.Date_of_Journey,dayfirst=True),
                dep_time=lambda df_: pd.to_datetime(df_.dep_time,format='mixed').dt.time,
                arrival_time=lambda df_: pd.to_datetime(df_.arrival_time,format='mixed').dt.time,
                duration=lambda df_:df_.duration.pipe(convert_to_minutes),
                total_stops=lambda df_: (
                    df_
                    .total_stops
                    .replace("non-stop","0")
                    .str.replace(" stops?","",regex=True)  # serarch for " stop" or " stops"
                    .pipe(lambda ser: pd.to_numeric(ser))
                ),
                additional_info=lambda df_: df_.additional_info.replace("No info","No info")
    )
        .drop(columns='route')
)


In [164]:
flights_cleaned=clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No info,12648


# 6.Split the Data

# 7.Export the Subsets